# GraphRAG Relevance, Factuality and Synthesis Evaluation

## Step 0: environment set up

In [1]:
from dotenv import load_dotenv
import os
from langchain_neo4j import Neo4jGraph
from libs import create_vector_index
import pandas as pd
from conn import connect2Googlesheet
from libs import context_builder

# Force reload of the .env file
load_dotenv()

True

In [2]:
# Connect to Neo4j database
try:
    graph = Neo4jGraph(
        url=os.getenv("NEO4J_URL"),
        username=os.getenv("NEO4J_USERNAME"),
        password=os.getenv("NEO4J_PASSWORD"),
    )
    print("Connected to Neo4j database successfully.")
except ValueError as e:
    print(f"Could not connect to Neo4j database: {e}")

Connected to Neo4j database successfully.


## Step 1: Create vector index

In [3]:
create_vector_index(graph, "entities")

✅ Index 'entities' already exists with correct dimensions: 384


## Step 2: Load questions from retrieve result doc and relevant doc

In [7]:
spreadsheet = connect2Googlesheet()

# Select the worksheet: relevance
worksheet = spreadsheet.get_worksheet(1)

# Get all records as a list of dictionaries
data = worksheet.get_all_records()

# Convert to Pandas DataFrame
df_MedQ = pd.DataFrame(data)
df_MedQ.head(10)

,condition,number,docs,Question,Mahmud's Note,status,comments,
0,ARDS,1,ACURASYS,Does early administration of neuromuscular blocking agents increases the ventilator free days?,Like,,,
1,ARDS,2,ACURASYS,Do patients with severe ARDS being treated with neuromuscular blocking agents have increased muscle weakness?,Replace,fixed,,
2,ARDS,3,ROSE,"In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?","Maybe this question: In patients with moderate to severe ARDS, does early use of continuous neuromuscular blockade improve mortality?",fixed,,
3,ARDS,4,ROSE,Do patients with moderate-to-severe ARDS have a significance difference in mortality rate between patients who recieved an early and continuous cisatracurium infusion than those with usual care approach with lighter sedation targets?,Local question (not sure if this is the aim of your project) It will be nice as second step after proving the general summarization is working but focusing in general summarization would be priority in my opinion so you can have meanigful tool.,fixed,Wrong concept since PEEP by itself is mandatory component in ventilator.,"Does the use of neuromuscular blockers in patients with moderate-to-severe ARDS impact cardiovascular stability, particularly in terms of vasopressor requirements and hemodynamic effects, compared to sedation strategy without routine neuromuscular blockade?"
4,ARDS,5,FACTT,"Among patients with ALI/ARDS, does a conservative fluid management strategy improves lung function and decrease ventilator days compared to liberal strategy?","Local question (not sure if this is the aim of your project) consider (WikiJournal): In patients with ALI/ARDS that are intubated and receiving positive pressure ventilation, how does the conservative compare to the liberal fluid management strategy in reducing mortality?",fixed,Check if studies defined conservative by CVP < 4 or elese just dont mention how much the CVP (i prefer the last approach),
5,ARDS,6,FACTT,Does a conservative fluid management strategy result in a significant difference in mortality as the primary outcome?,Local question,fixed,Does a conservative fluid management strategy result in a significant difference in mortality as the primary outcome?,
6,ARDS,7,ARDSNet,Does mechanical ventilation with a lower tidal volume result in decreased mortality with acute lung injury and acute respiratory distress syndrome?,Like (just make it as question),fixed,Maybe make it two questions. Better not combine two outcomes together so in case they are in opposite directions. Or make it neutral question like what is the effect of low TV ventilation on mortality and ventilator days,trying to make it yes/no questions
7,ARDS,8,ARDSNet,Does mechanical ventilation with a lower tidal volume result increased ventilator free days with patients with acute lung injury and acute respiratory distress syndrome?,Local,fixed,Wrong question. Just make it as second point for the question above so one about mortality and one about ventilator days,
8,ARDS,9,PROSEVA,Prone positioning sessions in patients with severe ARDS significantly decreased mortality?,Like,,,
9,ARDS,10,OSCILLATE,Does early application of high frequency oscillatory ventilation compared with ventilation strategy of low tidal volume decrease mortality?,Like,,,


## Step 3: Relevance check for top K questions

In [8]:
def retrieval_rel_docs(graph, questions, top_k=5):
    top_k_questions = questions.head(top_k)
    # Initialize a list to store the results
    results = []
    # Iterate over the top k questions
    for index, row in top_k_questions.iterrows():
        question_number = index + 1  # Assuming the question number is the index + 1
        question = row[
            "Question"
        ]  # Replace 'Question' with the actual column name for questions in df_MedQ

        # Generate response for the question
        context = context_builder(graph, question, method="vector")

        # Extract relevant documents from the response content
        # docs = response.choices[0].message.content  # Adjust this based on the actual response structure

        # Append the result to the list
        results.append(
            {
                "Question number": question_number,
                "Question": question,
                "Retrieved": context,
            }
        )

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(
        results, columns=["Question number", "Question", "Retrieved"]
    )

    return results_df


# retrieval_rel_docs (graph, questions, top_k=1)

In [9]:
# Set pandas display options to show the full text content
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
topk = 36
results_df = retrieval_rel_docs(graph, df_MedQ, top_k=topk)
results_df
results_df.to_csv("./outputs/retrieved_docs_results.csv", index=False)

KeyboardInterrupt: 

In [8]:
# old results
old_results = pd.read_csv("retrieved_docs_results.csv")
old_results

,Question number,Generated Docs
0,1,"""ACURASYS"""
1,2,"""ACURASYS"""
2,3,"""ACURASYS"""
3,4,"""ROSE"", ""ACURASYS"""
4,5,"""ARDS"", ""ACURASYS"""


In [15]:
# load the relevant documents
topk = 2
results_df = pd.read_csv("retrieved_docs_results.csv")
relevant_docs_df = pd.read_csv("relevant_docs.csv").rename(
    columns={"Docs": "Relevant Docs"}
)
concatenated_df = get_concatenate_df(results_df, relevant_docs_df, topk)
concatenated_df.head()

,Question,Relevant Docs,Generated Docs
0,Relevace Q1,ACURASYS ROSE,ACURASYS
1,Relevace Q2,ACURASYS ROSE,ACURASYS


In [16]:
relevant_docs_df.head()
query = "Find the most relevant document for critical care research"
context_builder(graph, query, method="vector")
relevant_docs_df.head()

,Unnamed: 0,Question,Relevant Docs
0,0,Relevace Q1,"ACURASYS, ROSE"
1,1,Relevace Q2,"ACURASYS, ROSE"
2,2,Relevace Q3,ROSE
3,3,Relevace Q4,ROSE
4,4,Relevace Q5,FACTT


In [14]:
# Display the concatenated DataFrame with metrics
concatenated_df = apply_metric(concatenated_df)
concatenated_df
# Save the concatenated DataFrame to a CSV file
# concatenated_df.to_csv('./outputs/concatenated_results.csv', index=False)

,Question,Relevant Docs,Generated Docs,Accuracy,Precision,Recall
0,Relevace Q1,ACURASYS ROSE,ACURASYS,0.5,1.0,0.5
1,Relevace Q2,ACURASYS ROSE,ACURASYS,0.5,1.0,0.5
